In [1]:
import timm
import math
import torch
import torch.nn as nn
import torchvision
from typing import Optional

C:\Users\16213\PycharmProjects\learn_cuda\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 原始
# ok

In [2]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DTYPE = torch.float32

In [4]:
BATCH_SIZE = 75

epochs = 40

lr = 0.01
momentum = 0.9

In [5]:
transform1 = torchvision.transforms.Compose([
                                                torchvision.transforms.RandomCrop(32, padding=4),
                                                torchvision.transforms.Resize(32),
                                                torchvision.transforms.RandomHorizontalFlip(),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                                torchvision.transforms.ConvertImageDtype(DTYPE)
                                             ])

transform2 = torchvision.transforms.Compose([
                                                torchvision.transforms.Resize(32),
                                                torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                                torchvision.transforms.ConvertImageDtype(DTYPE)
                                             ])

In [6]:
from torchvision.models.vision_transformer import Encoder

class VisionTransformer(nn.Module):
    """Vision Transformer as per https://arxiv.org/abs/2010.11929."""

    def __init__(
        self,
        image_size: int,
        patch_size: int,
        num_layers: int,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float = 0.0,
        attention_dropout: float = 0.0,
        num_classes: int = 10,
    ):
        super().__init__()
        if image_size % patch_size != 0:
            print("Input shape indivisible by patch size!")

        self.image_size = image_size
        self.patch_size = patch_size
        self.hidden_dim = hidden_dim
        self.mlp_dim = mlp_dim
        self.attention_dropout = attention_dropout
        self.dropout = dropout
        self.num_classes = num_classes


        self.conv_proj = nn.Conv2d(
            in_channels=3, out_channels=hidden_dim, kernel_size=patch_size, stride=patch_size
        )

        seq_length = (image_size // patch_size) ** 2

        # Add a class token
        self.class_token = nn.Parameter(torch.zeros(1, 1, hidden_dim))
        seq_length += 1

        self.encoder = Encoder(
            seq_length,
            num_layers,
            num_heads,
            hidden_dim,
            mlp_dim,
            dropout,
            attention_dropout,
        )
        self.seq_length = seq_length

        self.heads = nn.Sequential(
            nn.Linear(hidden_dim, num_classes)
        )

        fan_in = self.conv_proj.in_channels * self.conv_proj.kernel_size[0] * self.conv_proj.kernel_size[1]
        nn.init.trunc_normal_(self.conv_proj.weight, std=math.sqrt(1 / fan_in))
        nn.init.zeros_(self.conv_proj.bias)



    def _process_input(self, x: torch.Tensor) -> torch.Tensor:
        n, _, h, w = x.shape
        p = self.patch_size
        # torch._assert(h == self.image_size, f"Wrong image height! Expected {self.image_size} but got {h}!")
        # torch._assert(w == self.image_size, f"Wrong image width! Expected {self.image_size} but got {w}!")
        n_h = h // p
        n_w = w // p

        x = self.conv_proj(x)
        x = x.reshape(n, self.hidden_dim, n_h * n_w)
        x = x.permute(0, 2, 1)

        return x

    def forward(self, x: torch.Tensor):
        # Reshape and permute the input tensor
        x = self._process_input(x)
        n = x.shape[0]

        # Expand the class token to the full batch
        batch_class_token = self.class_token.expand(n, -1, -1)
        x = torch.cat([batch_class_token, x], dim=1)

        x = self.encoder(x)

        # Classifier "token" as used by standard language architectures
        x = x[:, 0]

        x = self.heads(x)

        return x

In [7]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=8, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=8, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
net = VisionTransformer(
    image_size=32,
    patch_size=8,
    num_layers=12,
    num_heads=12,
    hidden_dim=768,
    mlp_dim=768*4,
    dropout=0.2,
    attention_dropout=0.2,
    num_classes=10
)

In [9]:
net = net.train().to(device=DEVICE, dtype=DTYPE)

optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

loss_func = torch.nn.CrossEntropyLoss()

In [10]:
def tran(epoch):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    scheduler.step()
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))

In [11]:
def test():
    net.eval()
    all_counter=0
    correct_counter=0
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        out = net(inputs)
        out = out.detach().cpu().argmax(1)
        t = labels.cpu()
        for m in range(len(t)):
            all_counter += 1
            if t[m] == out[m]:
                correct_counter += 1

    print(correct_counter, all_counter, correct_counter / all_counter)
    return (correct_counter / all_counter)

In [12]:
correctRate = 0
for i in range(epochs):
    tran(i)
    r = test()
    if(r > correctRate):
        correctRate = r
        print("best: ", r , " in NO: ", i)
        torch.save(net.cpu(),"checkpoint/trans_vit2.pth")
        net = net.to(DEVICE)

[1] loss: 1366.490
2699 10000 0.2699
best:  0.2699  in NO:  0
[2] loss: 1153.591
3744 10000 0.3744
best:  0.3744  in NO:  1
[3] loss: 1088.261
3881 10000 0.3881
best:  0.3881  in NO:  2
[4] loss: 1046.649
4046 10000 0.4046
best:  0.4046  in NO:  3
[5] loss: 1010.137
4265 10000 0.4265
best:  0.4265  in NO:  4
[6] loss: 982.708
4705 10000 0.4705
best:  0.4705  in NO:  5
[7] loss: 953.450
4745 10000 0.4745
best:  0.4745  in NO:  6
[8] loss: 936.892
4802 10000 0.4802
best:  0.4802  in NO:  7
[9] loss: 912.539
4811 10000 0.4811
best:  0.4811  in NO:  8
[10] loss: 900.199
5138 10000 0.5138
best:  0.5138  in NO:  9
[11] loss: 881.858
5244 10000 0.5244
best:  0.5244  in NO:  10
[12] loss: 865.569
5160 10000 0.516
[13] loss: 850.073
5228 10000 0.5228
[14] loss: 835.203
5420 10000 0.542
best:  0.542  in NO:  13
[15] loss: 823.145
5547 10000 0.5547
best:  0.5547  in NO:  14
[16] loss: 806.852
5555 10000 0.5555
best:  0.5555  in NO:  15
[17] loss: 799.615
5667 10000 0.5667
best:  0.5667  in NO:  1

KeyboardInterrupt: 